In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split

import pandas as pd
import seaborn as sns

In [18]:
df = pd.read_csv('data/archive/healthcare-dataset-stroke-data.csv')
df.drop('id', inplace=True, axis=1)
df.dropna(inplace=True, subset=['bmi'])

# df = df[df["smoking_status"] != "Unknown"]
# x = df[df["stroke"] == 1]


X = df.drop('stroke', axis=1)
y = df["stroke"]
# df.info()

from sklearn.preprocessing import LabelEncoder

number = LabelEncoder()

df['gender'] = number.fit_transform(df['gender'].astype(str))

df["gender"].unique()
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,1,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,0,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,0,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,1,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5104,0,13.0,0,0,No,children,Rural,103.08,18.6,Unknown,0
5106,0,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,0,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,1,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=32)
print(f"Tamanho do conjunto de treino: {len(X_train)}")
print(f"Tamanho do conjunto de teste: {len(X_test)}")

Tamanho do conjunto de treino: 3436
Tamanho do conjunto de teste: 1473


In [4]:
pipeline = Pipeline(steps=[
    ("normalizacao", MaxAbsScaler()),  
    ("MLP", MLPClassifier())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
y_pred_prob = pipeline.predict_proba(X_test)
print(f"Acurácia de treinamento: {pipeline.score(X_train, y_train)}")

ValueError: could not convert string to float: 'Female'